In [11]:
import json, os, sys, pickle
import pandas as pd
import numpy as np 
from collections import defaultdict

In [12]:
with open('../../../../notebooks/celloracle_links.pkl', 'rb') as file:
    links_dict = pickle.load(file)

len(links_dict.keys())

6

In [13]:
links_dict['B-cell']

,source,target,coef_mean,coef_abs,p,-logp
0,Nfatc4,0610030E20Rik,-0.000207,0.000207,1.500551e-08,7.823749
1,Pax5,0610030E20Rik,0.000400,0.000400,4.740620e-02,1.324165
2,Ebf1,0610030E20Rik,0.000875,0.000875,1.625185e-09,8.789097
3,Atf3,0610030E20Rik,-0.000473,0.000473,2.122179e-12,11.673218
4,Mga,0610030E20Rik,0.000392,0.000392,3.169740e-10,9.498976
...,...,...,...,...,...,...
111160,Klf4,Zwilch,0.000019,0.000019,1.872425e-01,0.727596
111161,Zscan22,Zwilch,-0.000056,0.000056,1.562141e-04,3.806280
111162,Id1,Zwilch,0.000039,0.000039,5.493913e-03,2.260118
111163,Id2,Zwilch,0.000318,0.000318,9.703121e-06,5.013089


In [14]:
genes = links_dict['B-cell'].target.unique()
len(genes)

2374

In [15]:
df1 = links_dict['B-cell'].query('source=="Pax5"')
df2 = links_dict['Cd8 T-cell'].query('source=="Pax5"')

In [16]:
pd.DataFrame(
    [df2.set_index('target').loc[np.intersect1d(
        df1.target, df2.target)].coef_mean-df1.set_index('target').loc[
     np.intersect1d(df1.target, df2.target)].coef_mean]).T.sort_values(by='coef_mean', ascending=False)[:20]

,coef_mean
target,
Lyz2,0.064251
H2-K1,0.042472
Npm1,0.041504
Cox4i1,0.040964
Pomp,0.035392
Rps12,0.034086
Cd22,0.030748
Calm1,0.030058
B2m,0.029126


In [17]:
import numpy as np
import matplotlib.pyplot as plt

def generate_positions(radius_min, radius_max, ncells=200):
    positions = []
    for i in range(ncells):
        angle = np.random.uniform(0, 2 * np.pi)
        r = np.random.uniform(radius_min, radius_max)
        x = r * np.cos(angle)
        y = r * np.sin(angle)
        positions.append((x, y))
    return positions

# Can change this later
cluster_radii = {r: (r,r+1) for r, cluster in enumerate(links_dict.keys())}
cluster_radii

{0: (0, 1), 1: (1, 2), 2: (2, 3), 3: (3, 4), 4: (4, 5), 5: (5, 6)}

In [ ]:
def beta_func(x, y, c, tf_index): 
    x = np.array(x).flatten()
    y = np.array(y).flatten()
    c = np.array(c).flatten()
    tf_index = np.array(tf_index).flatten()
    
    return c * x + tf_index - y


In [67]:
positions = []

for cluster, cell_indices in cluster_index_dict.items():
    radii = cluster_radii[cluster]
    poses = generate_positions(radii[0], radii[1], ncells=len(cell_indices))
    x_pos = np.array([pos[0] for pos in poses])
    y_pos = np.array([pos[1] for pos in poses])
    labels = [cluster] * len(cell_indices)

    betas = np.array([beta_func(x_pos, y_pos, labels, tf) for tf in range(ntfs)])

    cluster_df = np.column_stack([labels, x_pos, y_pos, betas])
    positions.append(cluster_df)

positions = np.vstack(positions)
df = pd.DataFrame(all_positions, columns=['cluster', 'x', 'y'] + [f'beta_{i}' for i in range(betas_array.shape[0])])
df

KeyError: 6

In [ ]:
def beta_func(x, y, c, tf_index): 
    return c * x + tf_index - y

df['beta'] = df.apply(lambda row: row['A'] + row['B'], axis=1)

In [ ]:
df.to_csv('positions.csv')